In [1]:
from typing import Any, Tuple, Optional

import numpy as np
import torch
import torch.nn.functional as F
from torch.amp.autocast_mode import autocast
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchmetrics.classification import MultilabelAveragePrecision
from tqdm import tqdm
import wandb

wandb.init()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tqzhang04 (acoustic-species-identification). Use `wandb login --relogin` to force relogin


In [2]:
import pyha_analyzer as pa
from pyha_analyzer import config
from pyha_analyzer.dataset import get_datasets, make_dataloaders, PyhaDFDataset
from pyha_analyzer.utils import set_seed
from pyha_analyzer.models.early_stopper import EarlyStopper
from pyha_analyzer.models.timm_model import TimmModel
from pyha_analyzer.train import inference_valid, set_seed, run_batch, map_metric
import pandas as pd

In [3]:
tqdm.pandas()
cfg = config.cfg

In [4]:
# data = pd.read_csv("~./AM1-piha-tm-results.csv")
# data['FOLDER'] = ['/mnt/passive-acoustic-biodiversity/Peru_2019_Audiomoth_Sound_Recordings/GRABADOR-SDZG-AM-1'] * data.shape[0]
#data.to_csv("~./AM1-piha-tm-results-2.csv")
data_chunked = pd.read_csv("~./Modified_AM1-fly-tm-results-labelled-chunked.csv")
onehot = pd.get_dummies(data_chunked['MANUAL ID'])
infer_data = pd.concat([data_chunked, onehot], axis=1)
infer_data

,Unnamed: 0,FOLDER,IN FILE,CLIP LENGTH,CHANNEL,OFFSET,DURATION,SAMPLE RATE,MANUAL ID,CONFIDENCE,FLY_0,FLY_1
0,0,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190617_073000.WAV,60.0,0,27.0,3.0,44100,FLY_0,0.311595,True,False
1,1,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190617_073000.WAV,60.0,0,30.0,3.0,44100,FLY_0,0.311595,True,False
2,2,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190617_102000.WAV,60.0,0,27.0,3.0,44100,FLY_0,0.321725,True,False
3,3,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190617_102000.WAV,60.0,0,30.0,3.0,44100,FLY_0,0.321725,True,False
4,4,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190617_045000.WAV,60.0,0,3.0,3.0,44100,FLY_0,0.305411,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1966,1966,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190705_175000.WAV,60.0,0,3.0,3.0,44100,FLY_0,0.318997,True,False
1967,1967,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190705_175000.WAV,60.0,0,6.0,3.0,44100,FLY_0,0.318997,True,False
1968,1968,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190705_175000.WAV,60.0,0,9.0,3.0,44100,FLY_0,0.365394,True,False
1969,1969,/mnt/passive-acoustic-biodiversity/Peru_2019_A...,20190705_175000.WAV,60.0,0,12.0,3.0,44100,FLY_0,0.365394,True,False


In [5]:
infer_data.to_csv("~./Modified_AM1-fly-tm-results-labelled-infer.csv")

In [6]:
#logger.info("Device is: %s, Preprocessing Device is %s", cfg.device, cfg.prepros_device)
set_seed(cfg.seed)

# Load in dataset
#logger.info("Loading Dataset...")
train_dataset, val_dataset, infer_dataset = get_datasets(cfg)
train_dataloader, val_dataloader, infer_dataloader = make_dataloaders(
    train_dataset, val_dataset, infer_dataset, cfg
)

100%|██████████| 344/344 [00:00<00:00, 3946.74it/s]
/home/t.zhang/.venv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(
100%|██████████| 35/35 [00:00<00:00, 2711.05it/s]
/home/t.zhang/.venv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(
100%|██████████| 379/379 [00:00<00:00, 3834.60it/s]
/home/t.zhang/.venv/lib/python3.10/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


In [7]:
#logger.info("Loading Model...")
model_for_run = TimmModel(num_classes=train_dataset.num_classes,
                            model_name=cfg.model).to(cfg.device)
model_for_run.create_loss_fn(train_dataset)
#if cfg.model_checkpoint != "":
#model_for_run.load_state_dict(torch.load("first_report/eca_nfnet_l0-20240220-2320_human.pt"))
#model_for_run.load_state_dict(torch.load("/home/thesis-generator/acoustic-multiclass-training/reduced_uniform_extra/eca_nfnet_l0-20240416-0532_human_reduced_uniform_extra.pt"))
#model_for_run.load_state_dict(torch.load("concat_train_experiments/eca_nfnet_l0-20240228-2236_fgbg_concat.pt"))
#model_for_run.load_state_dict(torch.load("pyha_analyzer/models/eca_nfnet_l0-20240423-2354_w_other.pt"))
model_for_run.load_state_dict(torch.load("eca_nfnet_l0-20240422-0057.pt"))
#tf_efficientnet_b4-20240420-2025.pt

<All keys matched successfully>

In [8]:
optimizer = Adam(model_for_run.parameters(), lr=cfg.learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, eta_min=1e-5, T_max=10)

#logger.info("Training...")
early_stopper = EarlyStopper(patience=cfg.patience, min_delta=cfg.min_valid_map_delta)

best_valid_cmap = 0.0

# valid_cmap, best_valid_cmap = valid(model_for_run,
#                                     val_dataloader,
#                                     infer_dataloader,
#                                     1.0,
#                                     best_valid_cmap)



In [9]:
model_for_run.eval()
model = model_for_run
data_loader = infer_dataloader


log_pred, log_label = [], []

num_valid_samples = int(len(data_loader))

# tqdm is a progress bar
dl_iter = tqdm(data_loader, position=5, total=num_valid_samples)


In [10]:
with torch.no_grad():
    for _, (mels, labels) in enumerate(dl_iter):
        _, outputs = run_batch(model, mels, labels)
        log_pred.append(torch.clone(outputs.cpu()).detach())
        log_label.append(torch.clone(labels.cpu()).detach())

# sigmoid predictions
log_pred = F.sigmoid(torch.cat(log_pred)).to(cfg.device)

dataset: PyhaDFDataset = data_loader.dataset # type: ignore
cmap, smap = map_metric(log_pred, torch.cat(log_label), dataset.class_dist)
# Log to Weights and Biases

torch.save(log_pred, 'big_run_models/predictions_big_run_fgbg.pt')
torch.save(log_label, 'big_run_models/labels_big_run_fgbg.pt')
print(cmap, smap)

/home/t.zhang/.venv/lib/python3.10/site-packages/timm/layers/std_conv.py:102: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(x, weight, self.bias, self.stride, self.padding, self.dilation, self.groups)
























































































































































































































































100%|██████████| 62/62 [10:28<00:00, 10.14s/it]

0.556395947933197 0.7681124210357666


In [12]:
log_pred

tensor([[0.7032, 0.8907],
        [0.9787, 0.7635],
        [0.4700, 0.9494],
        ...,
        [0.4354, 0.9932],
        [0.2754, 0.9990],
        [0.3874, 0.9984]], device='cuda:0')

In [11]:
log_label

[tensor([[0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.]]),
 tensor([[0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [1., 0.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [0., 1.],
         [1., 0.],
         [1., 0.],
         [1., 0.],
         [1., 0.],
         [1., 0.],
         [1., 0.],
         [

In [ ]:
attila_log_pred = torch.tensor(log_pred)

/tmp/ipykernel_636154/1368344336.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attila_log_pred = torch.tensor(log_pred)


In [ ]:
torch.sum((log_pred[:, 0] < log_pred[:, 1]).float())

tensor(1068., device='cuda:0')

In [ ]:
attila_cmap, attila_smap = 0.49828091263771057, 0.9878062009811401
attila_cmap_peru, attila_smap_peru = 0.3933272957801819, 0.4456251263618469
piha_cmap, piha_smap = 0.766303, 0.788464
piha_smap_peru, pira_cmap_peru = 0.8015578985214233, 0.8001036643981934

In [ ]:
results = pd.DataFrame({'CmAP': [attila_cmap, piha_cmap],
                        'SmAP': [attila_smap, piha_smap],
                        'Species': ['Attila', 'Piha']}).melt('Species')
results

,Species,variable,value
0,Attila,CmAP,0.498281
1,Piha,CmAP,0.766303
2,Attila,SmAP,0.987806
3,Piha,SmAP,0.788464
